In [ ]:
from collections import Counter
import requests
import pandas as pd
import time
import os


In [ ]:
#  Инициализация фунции для сентимент-анализа упоминания  компаниии с использованием API к модели YandeXGPT
def baseline_sentiment(sentence: str, company: str,   retries: int = 5, delay: int = 1, model: str = 'lite') -> str | None:
    """
    Классифицирует тональность предложения с упоминанием компании, возвращая одно из значений: 'negative', 'neutral' или 'positive'.

    Параметры:
    - sentence (str): Предложение, в котором необходимо оценить тональность.
    - company (str): Название компании, упоминание которой оценивается.
    - retries (int, по умолчанию 5): Количество попыток при неудачном запросе.
    - delay (int, по умолчанию 1): Время задержки между повторными попытками в секундах.
    - model (str, по умолчанию 'lite'): Тип модели для использования — 'lite' для упрощённой версии или 'pro' для полной версии.

    Возвращаемое значение:
    - str: Одно из значений 'negative', 'neutral' или 'positive', если запрос выполнен успешно.
    - None: Если произошла ошибка или количество попыток исчерпано.
    """

    # URL для обращения к модели Yandex GPT
    URL = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
    # Получаем API ключ и ID папки из переменных окружения
    api_key = os.getenv("API_KEY")
    folder_id = os.getenv("FOLDER_ID")

    # Инициализируем пустой словарь для данных запроса
    data = {}

    # Выбор модели в зависимости от переданного параметра
    if model == 'lite':
        data["modelUri"] = f"gpt://{folder_id}/yandexgpt-lite/latest"  # lite-версия модели
    elif model == 'pro':
        data["modelUri"] = f"gpt://{folder_id}/yandexgpt/latest"  # pro-версия модели
    else:
        print("Неправильная модель, доступные модели 'lite' и 'pro'")
        return None  # Завершаем выполнение функции, если модель указана неверно

    # Настройка опций генерации
    data["completionOptions"] = {"temperature": 0.3, "maxTokens": 500}  # Температура и максимальное количество токенов

    # Простой системный промт для модели с инструкцией по оценке тональности
    system_prompt = f"""Оцени тональность упоминания компании {company}, ответь одним словом negative, neutral или positive.
    Запомни: в ответе должно быть только одно слово - negative, neutral или positive, без пояснений и дополнительного текста."""

    # Формируем сообщения для передачи в модель (системный и пользовательский контекст)
    data["messages"] = [
        {"role": "system", "text": system_prompt},  # Системная инструкция
        {"role": "user", "text": sentence}  # Текст, который пользователь хочет классифицировать
    ]

    # Цикл для повторных попыток запроса в случае ошибки или превышения лимитов
    for attempt in range(retries):
        try:
            # Отправляем POST-запрос к API Yandex GPT с данными и заголовками
            response = requests.post(
                URL,
                headers={
                    "Accept": "application/json",
                    "Authorization": f"Api-Key {api_key}"  # Авторизация с использованием API ключа
                },
                json=data,  # Передаем данные запроса
            ).json()

            # Возвращаем результат — одно слово с оценкой тональности, переведенное в нижний регистр
            return response["result"]["alternatives"][0]['message']['text'].lower().rstrip(".")

        except Exception as e:
            # Проверяем, если произошла ошибка лимита запросов (HTTP 429)
            if response.get('error', {}).get('httpCode') == 429:
                print(f"Превышен лимит запросов. Попытка {attempt + 1} из {retries}. Ожидание {delay} секунд.")
                time.sleep(delay)  # Ждем перед новой попыткой
            else:
                # Любая другая ошибка выводится в консоль
                print(f"Произошла ошибка: {e}")
                return None  # Возвращаем None при ошибке

    # Если все попытки исчерпаны, выводим сообщение
    print("Превышено количество попыток.")
    return None  # Возвращаем None, если все попытки завершились неудачно


In [ ]:
# Импорт данных из файла
data = pd.read_excel('ru_data_test.xlsx')
data.head(2)

,sentence,object,tonality
0,Схему обнаружили аналитики департамента Digita...,Digital Risk Protection,positive
1,Лидеры ВОГ не избавлялись от непрофильных акти...,ВОГ,negative


# Baseline

In [ ]:
# Добавим в dataframe столбец с результатами функции baseline_sentiment()
data['sentiment-lite'] = data.apply(lambda row: baseline_sentiment(row['sentence'], row['object'],   model='lite'), axis=1)

In [ ]:
# Оценка точность модели
accuracy =len(data[data['sentiment-lite'] == data['tonality']])/len(data)
print(f"Точность lite модели: {accuracy}")

Точность lite модели: 0.6306306306306306


In [ ]:
# Позиции с ошибочными оценками
data[data['sentiment-lite'] != data['tonality']]

,sentence,object,tonality,sentiment-lite
0,Схему обнаружили аналитики департамента Digita...,Digital Risk Protection,positive,neutral
4,"В январе в Microsoft заявили, что киберпреступ...",Microsoft,neutral,negative
5,Атака NGC2180 была обнаружена специалистами Со...,Солар,positive,в интернете есть много сайтов с информацией на...
6,"ИБ компания Proofpoint обнаружила, что группир...",Proofpoint,positive,в интернете есть много сайтов с информацией на...
10,Специалисты FortiGuard Labs выявили новый вари...,FortiGuard Labs,positive,neutral
14,"ИБ компания GuidePoint Security обнаружила, чт...",GuidePoint Security,positive,negative
21,Также в марте Вкусвилл получил 80% в компании ...,Вкусвилл,positive,neutral
24,Softline через свою 100% ную дочку Скай Технол...,Softline,positive,neutral
25,"Скай Технолоджис , на 100% дочерняя ГК Softlin...",Скай Технолоджис,positive,neutral
26,Словацкая компания ESET обнаружила деятельност...,ESET,positive,neutral


  Анализ ошибок модели показывает что, модель часто выдает предсказание neutral в место positive и наоборот. Также на часть запросов модель предлагает поиск в интернете, большенство и зтаких запросов имеют лабел negative и вероятно такой ответ связан с цензурой. В других эксперементах модель отказывалась давать ответ выдавая сообщение указывающее на цензуру.
  Так же проведенные эксперементы показали что нет существенной разницы в ответах lite и pro моделей


In [ ]:

def base_classifier(sentence: str, company: str, retries: int = 5, delay: int = 1) -> str:
    """
    Функция для классификации тональности упоминания компании на базе YandexGPT.

    Параметры:
    sentence (str): Текст, который нужно проанализировать.
    company (str): Название компании, для которой анализируется тональность упоминания.
    retries (int, optional): Количество попыток при возникновении ошибок или превышении лимита запросов. По умолчанию 5.
    delay (int, optional): Время задержки перед повторной попыткой (в секундах). По умолчанию 1.

    Возвращает:
    str: Наиболее вероятная тональность из возможных ("positive", "negative", "neutral").
         Возвращает None, если не удается получить результат.
    """
    URL = "https://llm.api.cloud.yandex.net/foundationModels/v1/fewShotTextClassification"

    # Задержка перед началом выполнения (если требуется)
    time.sleep(delay)

    # Получаем API-ключ и ID папки из переменных окружения
    api_key = os.getenv("API_KEY")
    folder_id = os.getenv("FOLDER_ID")

    # Собираем запрос с параметрами для классификации
    data = {
        "modelUri": f"cls://{folder_id}/yandexgpt/latest",
        "taskDescription": f"Оцени тональность упоминания компании {company}.",
        "labels": ["positive", "negative", "neutral"],
        "text": sentence
    }

    # Попытка выполнить запрос до указанного числа retries
    for attempt in range(retries):
        try:
            # Отправляем POST-запрос на сервер YandexGPT
            response = requests.post(
                URL,
                headers={
                    "Accept": "application/json",
                    "Authorization": f"Api-Key {api_key}"
                },
                json=data,
            ).json()  # Ожидаем, что ответ придёт в формате JSON

            # Проверяем наличие поля 'predictions' в ответе
            if 'predictions' in response:
                # Выбираем метку с наибольшей уверенностью
                best_prediction = max(response['predictions'], key=lambda x: x['confidence'])
                best_label = best_prediction['label']
                return best_label

            # Если 'predictions' не найдено, выводим сообщение об ошибке
            print(f"Не найдено поле 'predictions' в ответе: {response}")
            return None

        except Exception as e:
            # Если возникает ошибка лимита запросов (например, код 8), повторяем попытку
            if isinstance(response, dict) and response.get('code') == 8:
                print(f"Превышен лимит запросов: попытка {attempt + 1} из {retries}. Ожидание {delay} секунд.")
                time.sleep(delay)  # Задержка перед новой попыткой
            else:
                # Если возникает другая ошибка, выводим её и завершаем выполнение
                print(f"Произошла ошибка: {e}")
                return None

    # Если все попытки исчерпаны, выводим сообщение
    print("Превышено количество попыток.")
    return None

# Пример использования
sentence = "Лидеры ВОГ не избавлялись от непрофильных активов в Вологде, Казани, Липецке, Новосибирске, Омске и Калининградской области, а продавали их по заниженной цене, отметили правоохранители."
company = "ВОГ"
base_classifier(sentence, company)


'negative'

In [ ]:
# Добавляем в таблицу столбец с результатами слассификатора
data['sentiment-classifieer-label'] = data.apply(lambda row: base_classifier(row['sentence'], row['object']),
    axis=1,
 )

In [ ]:
# Оценка точность модели классификатора
accuracy =len(data[data['sentiment-classifieer-label'] == data['tonality']])/len(data)
print(f"Точность классификатора: {accuracy}")

Точность классификатора: 0.6216216216216216


In [ ]:
# Позиции с ошибочными оценками классификатора
data[data['sentiment-classifieer-label'] != data['tonality']]

,sentence,object,tonality,sentiment-lite,sentiment-classifieer-label
0,Схему обнаружили аналитики департамента Digita...,Digital Risk Protection,positive,neutral,neutral
2,Полиция задержала президента общества глухих С...,ВОГ,negative,negative,neutral
4,"В январе в Microsoft заявили, что киберпреступ...",Microsoft,neutral,negative,negative
5,Атака NGC2180 была обнаружена специалистами Со...,Солар,positive,в интернете есть много сайтов с информацией на...,neutral
6,"ИБ компания Proofpoint обнаружила, что группир...",Proofpoint,positive,в интернете есть много сайтов с информацией на...,neutral
9,Исследователи FortiGuard Labs выявили автора в...,FortiGuard Labs,positive,positive,negative
10,Специалисты FortiGuard Labs выявили новый вари...,FortiGuard Labs,positive,neutral,negative
14,"ИБ компания GuidePoint Security обнаружила, чт...",GuidePoint Security,positive,negative,negative
21,Также в марте Вкусвилл получил 80% в компании ...,Вкусвилл,positive,neutral,neutral
24,Softline через свою 100% ную дочку Скай Технол...,Softline,positive,neutral,neutral


Несмотря на то что accurecy моделей примерно равны, так как у классификатора нет цензуры и он дал ответы на все запросы, а liti модель часть часть запросов проигнорировала. Можно сделать вывод что на тех запросах где lite модель дает ответ ее качество выше чем у классификатора.


In [ ]:
#  Для позиций где небыл получен ответ от  lite модели примениним классификатор
valid_sentiments = ['negative', 'neutral', 'positive']

# Применяем условие и замену через apply()
data['sentiment-lite'] = data.apply(
    lambda row: row['sentiment-lite']
                if row['sentiment-lite'] in valid_sentiments
                else base_classifier(row['sentence'], row['object']),
    axis=1
)

In [ ]:
# Оценка accuracy полученных результатов
accuracy =len(data[data['sentiment-lite'] == data['tonality']])/len(data)
print(f"Точность lite модели с класификацей цензурированных запросов: {accuracy}")

Точность lite модели с класификацей цензурированных запросов: 0.6936936936936937


## Применение более формализованного промта

Для повышения качества анализа используем более сложный промт с формализацие классов

In [181]:
#  Инициализация фунции с более сложным промтом
def sentiment(sentence: str, company: str,   retries: int = 5, delay: int = 1, model: str = 'lite') -> str | None:
    """
    Классифицирует тональность предложения с упоминанием компании, возвращая одно из значений: 'negative', 'neutral' или 'positive'.

    Параметры:
    - sentence (str): Предложение, в котором необходимо оценить тональность.
    - company (str): Название компании, упоминание которой оценивается.
    - retries (int, по умолчанию 5): Количество попыток при неудачном запросе.
    - delay (int, по умолчанию 1): Время задержки между повторными попытками в секундах.
    - model (str, по умолчанию 'lite'): Тип модели для использования — 'lite' для упрощённой версии или 'pro' для полной версии.

    Возвращаемое значение:
    - str: Одно из значений 'negative', 'neutral' или 'positive', если запрос выполнен успешно.
    - None: Если произошла ошибка или количество попыток исчерпано.
    """

    # URL для обращения к модели Yandex GPT
    URL = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
    # Получаем API ключ и ID папки из переменных окружения
    api_key = os.getenv("API_KEY")
    folder_id = os.getenv("FOLDER_ID")

    # Инициализируем пустой словарь для данных запроса
    data = {}

    # Выбор модели в зависимости от переданного параметра
    if model == 'lite':
        data["modelUri"] = f"gpt://{folder_id}/yandexgpt-lite/latest"  # lite-версия модели
    elif model == 'pro':
        data["modelUri"] = f"gpt://{folder_id}/yandexgpt/latest"  # pro-версия модели
    else:
        print("Неправильная модель, доступные модели 'lite' и 'pro'")
        return None  # Завершаем выполнение функции, если модель указана неверно

    # Настройка опций генерации
    data["completionOptions"] = {"temperature": 0.3, "maxTokens": 500}  # Температура и максимальное количество токенов

    # Простой системный промт для модели с инструкцией по оценке тональности
    system_prompt = f"""Оцени тональность упоминания компании {company}, ответь одним словом negative, neutral или positive.

    Используй следующие критерии для определения тональности:

    POSITIVE тональность:
    - Рост финансовых показателей (выручка, прибыль)
    - Успешные сделки, поглощения, IPO
    - Расширение бизнеса, выход на новые рынки
    - Технологические достижения, инновации
    - Для ИБ-компаний: обнаружение угроз/уязвимостей
    - Получение патентов, сертификатов
    - Рост команды, открытие офисов
    - Позитивные назначения руководителей

    NEGATIVE тональность:
    - Падение финансовых показателей
    - Убытки, банкротство
    - Отзыв лицензий
    - Взломы систем компании
    - Утечки данных клиентов
    - Уязвимости в продуктах (кроме ИБ-компаний)
    - Судебные иски против компании
    - Обвинения в нарушениях
    - Санкции, уход с рынка

    NEUTRAL тональность:
    - Описание продуктов без оценки
    - Цитаты представителей компании
    - Упоминания в списках
    - Технические характеристики
    - Процессы внедрения
    - Планы и стратегии
    - Прогнозы без конкретных результатов
    Запомни: в ответе должно быть только одно слово - negative, neutral или positive, без пояснений и дополнительного текста."""

    # Формируем сообщения для передачи в модель (системный и пользовательский контекст)
    data["messages"] = [
        {"role": "system", "text": system_prompt},  # Системная инструкция
        {"role": "user", "text": sentence}  # Текст, который пользователь хочет классифицировать
    ]

    # Цикл для повторных попыток запроса в случае ошибки или превышения лимитов
    for attempt in range(retries):
        try:
            # Отправляем POST-запрос к API Yandex GPT с данными и заголовками
            response = requests.post(
                URL,
                headers={
                    "Accept": "application/json",
                    "Authorization": f"Api-Key {api_key}"  # Авторизация с использованием API ключа
                },
                json=data,  # Передаем данные запроса
            ).json()

            # Возвращаем результат — одно слово с оценкой тональности, переведенное в нижний регистр
            return response["result"]["alternatives"][0]['message']['text'].lower().rstrip(".")

        except Exception as e:
            # Проверяем, если произошла ошибка лимита запросов (HTTP 429)
            if response.get('error', {}).get('httpCode') == 429:
                print(f"Превышен лимит запросов. Попытка {attempt + 1} из {retries}. Ожидание {delay} секунд.")
                time.sleep(delay)  # Ждем перед новой попыткой
            else:
                # Любая другая ошибка выводится в консоль
                print(f"Произошла ошибка: {e}")
                return None  # Возвращаем None при ошибке

    # Если все попытки исчерпаны, выводим сообщение
    print("Превышено количество попыток.")
    return None  # Возвращаем None, если все попытки завершились неудачно


def classifier(sentence, company, retries=5, delay=1):
    URL = "https://llm.api.cloud.yandex.net/foundationModels/v1/fewShotTextClassification"
    time.sleep(delay)
    api_key = os.getenv("API_KEY")
    folder_id = os.getenv("FOLDER_ID")

    # Собираем запрос
    data = {}
    data["modelUri"] = f"cls://{folder_id}/yandexgpt/latest"
    data["taskDescription"] = f"""Оцени тональность упоминания компании {company}.

    Используй следующие критерии для определения тональности:

    POSITIVE тональность:
    - Рост финансовых показателей (выручка, прибыль)
    - Успешные сделки, поглощения, IPO
    - Расширение бизнеса, выход на новые рынки
    - Технологические достижения, инновации
    - Для ИБ-компаний: обнаружение угроз/уязвимостей
    - Получение патентов, сертификатов
    - Рост команды, открытие офисов
    - Позитивные назначения руководителей

    NEGATIVE тональность:
    - Падение финансовых показателей
    - Убытки, банкротство
    - Отзыв лицензий
    - Взломы систем компании
    - Утечки данных клиентов
    - Уязвимости в продуктах (кроме ИБ-компаний)
    - Судебные иски против компании
    - Обвинения в нарушениях
    - Санкции, уход с рынка

    NEUTRAL тональность:
    - Описание продуктов без оценки
    - Цитаты представителей компании
    - Упоминания в списках
    - Технические характеристики
    - Процессы внедрения
    - Планы и стратегии
    - Прогнозы без конкретных результатов"""

    data["labels"] = ["positive", "negative", "neutral"]
    data["text"] = sentence
    for attempt in range(retries):
        try:
            # Отправляем запрос
            response = requests.post(
                URL,
                headers={
                    "Accept": "application/json",
                    "Authorization": f"Api-Key {api_key}"
                },
                json=data,
            ).json()  # Ожидаем, что ответ будет JSON

            # Проверяем, что в ответе есть 'predictions'
            if 'predictions' in response:
                best_prediction = max(response['predictions'], key=lambda x: x['confidence'])
                best_label = best_prediction['label']
                return best_label

            # Если нет поля predictions, выводим ошибку
            print(f"Не найдено поле 'predictions' в ответе: {response}")
            return None

        except Exception as e:
            # Проверяем на лимит запросов
            if isinstance(response, dict) and response.get('code') == 8:
                print(f"Превышен лимит запросов: попытка {attempt + 1} из {retries}. Ожидание {delay} секунд.")
                time.sleep(delay)  # Задержка перед повторной попыткой
            else:
                print(f"Произошла ошибка: {e}")
                return None
    print("Превышено количество попыток.")
    return None


In [182]:
# Добавим в dataframe столбец с результатами функции sentiment()
data['sentiment'] = data.apply(lambda row: sentiment(row['sentence'], row['object'],   model='lite'), axis=1)

#  Для позиций где небыл получен ответ от  lite модели примениним классификатор
valid_sentiments = ['negative', 'neutral', 'positive']

data['sentiment'] = data.apply(
    lambda row: row['sentiment']
                if row['sentiment'] in valid_sentiments
                else classifier(row['sentence'], row['object']),
    axis=1
)

Превышен лимит запросов. Попытка 1 из 5. Ожидание 1 секунд.


In [183]:
# Оценка accuracy полученных результатов
accuracy =len(data[data['sentiment'] == data['tonality']])/len(data)
print(f"Точность с применением более точного промта: {accuracy}")

Точность с применением более точного промта: 0.8018018018018018


In [187]:
# Позиции с ошибочными оценками
data[data['sentiment'] != data['tonality']][['sentence','object','tonality','sentiment']]

,sentence,object,tonality,sentiment
4,"В январе в Microsoft заявили, что киберпреступ...",Microsoft,neutral,negative
14,"ИБ компания GuidePoint Security обнаружила, чт...",GuidePoint Security,positive,negative
26,Словацкая компания ESET обнаружила деятельност...,ESET,positive,negative
34,"Евгений присоединился к NGR Softlab в 2023 г.,...",NGR Softlab,positive,neutral
48,"Ранее, в 2021 г., специалисты Positive Technol...",Positive Technologies,positive,negative
52,"В 2022 году исследователи из компании Zscaler,...",Zscaler,positive,negative
53,Исследователи Dr Web обнаружили множество прил...,Dr Web,positive,negative
56,Игровой издатель и разработчик Astrum Entertai...,Astrum Entertainment,positive,neutral
66,"На фоне недавних президентских выборов в РФ, и...",F.A.C.C.T.,positive,negative
70,К Angara Security на позиции директора по упра...,Angara Security,positive,neutral
